In [8]:
using DataFrames
using JuMP, Gurobi
using Gadfly

In [108]:
m = Model(solver=GurobiSolver())

iGenLoad = readtable("BatteryModel_GenLoad.csv",header=true)
iPrices = readtable("BatteryModel_Prices.csv",header=true)
iThermalParameters = readtable("Thermal_cfs.csv",header=true) 
iThermalFactors = readtable("ComplexThermal_Factors.csv", header=true) 

T = length(iPrices[:pHour])
Tplus = T+2
pHours = iPrices[:pHour][1:T]

pBuyEnergy = iPrices[:pBuy_Energy][1:T]
pSellEnergy = iPrices[:pSell_Energy][1:T]
pNonControllableLoad = iGenLoad[:pNonControllableLoad][1:T]

pCf = iThermalParameters[:]
pTf = iThermalFactors[:]


#pTz = iThermalFactors[:Tz][1:Tplus]
#pTx = iThermalFactors[:Tx][1:Tplus]
#pQrad = iThermalFactors[:Qrad][1:Tplus]
#pQconv = iThermalFactors[:Qconv][1:Tplus]
#pQwind = iThermalFactors[:Qwindow_solar][1:Tplus]
#pQsurf = iThermalFactors[:Qsurface_solar][1:Tplus]
#pTground = iThermalFactors[:Tground][1:Tplus]

pCOP = 3
pDeadband = 2
pTempDevPenalty = 10
pSetpoint = 20


n=2 #thermal model order
k=6 #number of exogenous variables

6

In [107]:
#A = convert(Array,pTf[1:3,3:8])
#B = convert(Array,pCf[1:7,1:3])
#B = convert(Array,pCf[1:3,2:7])
#C = sum(A.*B)
#A[1:3,2:8]
#A

aTf = convert(Array,pTf[1:3,3:8])
aCf = convert(Array,pCf[1:3,2:7])
atmp_fct = convert(Array,pCf[1:3,1])
atmp_lag = convert(Array,pTf[1:3,2])
atmp_lag

3-element Array{Float64,1}:
 22.2644
 22.4039
 22.4099

In [111]:
##Noras Thermal Model:
#cCV= [((n+1)*k)+n] #constant coefficients - load from first tab in nora's spreadsheet
#cBTM = [T+n, k+1] #weather and build input data

@defVar(m, vTempInt[t=1:T])                   # deg C
@defVar(m, vTempLow[t=1:T])                   # deg C
@defVar(m, vTempHigh[t=1:T])                  # deg C
@defVar(m, vPowerHVAC[t=1:T])                 # watts
@defExpr(vQhvac[t=1:T], pCOP*vPowerHVAC[t])


#A = convert(Array,pTf[1:3,2:8])
atmp_fct = convert(Array,pCf[1:3,1])
aCf = convert(Array,pCf[1:3,2:7])

# determine temp for t = 1 to n, from lag variables
for t=1:n 
    @addConstraint(m, atmp_lag[t]==vTempInt[t])
    
    @addConstraint(m, (pSetpoint-pDeadband)-vTempLow[t]<=vTempInt[t])
    @addConstraint(m, (pSetpoint+pDeadband)+vTempHigh[t]>=vTempInt[t])

end

# determine temp for t > 1 from existing temp, hvac, and lag variables
for t=(n+1):T
    @addConstraint(m, sum(aCf.*convert(Array,pTf[t-2:t,3:8]))
                +atmp_fct[1]*vTempInt[t-1]+atmp_fct[2]*vTempInt[t-2]
    -sum(vPowerHVAC[(t-2):t].*convert(Array,pTf[t-2:t,5]))==vTempInt[t])
    # + atmp_fct[1]*vTempInt[t-1]+ atmp_fct[2]*vTempInt[t-2]
    
    @addConstraint(m, (pSetpoint-pDeadband)-vTempLow[t]<=vTempInt[t])
    @addConstraint(m, (pSetpoint+pDeadband)+vTempHigh[t]>=vTempInt[t])

end

@defExpr(vTotalTempDev[t=1:T],vTempHigh[t]+vTempLow[t])
@defExpr(TotalCost, sum{pBuyEnergy[t]*vPowerHVAC[t]/1000, t=1:T}+pTempDevPenalty*sum{vTotalTempDev[t],t=1:T})
@setObjective(m, Min, TotalCost)

4.5650000000000005e-5 vPowerHVAC[1] + 3.972e-5 vPowerHVAC[2] + 4.5400000000000006e-5 vPowerHVAC[3] + 4.0390000000000005e-5 vPowerHVAC[4] + 4.189e-5 vPowerHVAC[5] + 3.961e-5 vPowerHVAC[6] + 3.737e-5 vPowerHVAC[7] + 3.5770000000000005e-5 vPowerHVAC[8] + 3.47e-5 vPowerHVAC[9] + 3.303e-5 vPowerHVAC[10] + 3.8010000000000004e-5 vPowerHVAC[11] + 3.601e-5 vPowerHVAC[12] + 4.303e-5 vPowerHVAC[13] + 4.0680000000000004e-5 vPowerHVAC[14] + 5.0560000000000004e-5 vPowerHVAC[15] + 4.1349999999999995e-5 vPowerHVAC[16] + 3.7760000000000004e-5 vPowerHVAC[17] + 3.934e-5 vPowerHVAC[18] + 4.592e-5 vPowerHVAC[19] + 5.499e-5 vPowerHVAC[20] + 5.6529999999999996e-5 vPowerHVAC[21] + 7.126e-5 vPowerHVAC[22] + 5.051e-5 vPowerHVAC[23] + 5.5599999999999996e-5 vPowerHVAC[24] + 10 vTempHigh[1] + 10 vTempLow[1] + 10 vTempHigh[2] + 10 vTempLow[2] + 10 vTempHigh[3] + 10 vTempLow[3] + 10 vTempHigh[4] + 10 vTempLow[4] + 10 vTempHigh[5] + 10 vTempLow[5] + 10 vTempHigh[6] + 10 vTempLow[6] + 10 vTempHigh[7] + 10 vTempLow[7] 

In [112]:
status = solve(m)
status
println("Objective value: ", getObjectiveValue(m))

Optimize a model with 216 rows, 288 columns and 558 nonzeros
Coefficient statistics:
  Matrix range    [2e-01, 2e+03]
  Objective range [3e-05, 1e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [4e-01, 2e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s

Solved in 144 iterations and 0.00 seconds
Unbounded model


Objective value: NaN


In [ ]:
@addConstraint(m,vTempInt[t]== pCf[1]*vTempInt[t-1]+pCf[2]*vTempInt[t-2]                         #past zonal temps
        +pCf[3]*pTx[t]+pCf[4]*pTx[t-1]+pCf[5]*pTx[t-2]                      #current and past ambient temps
        +pCf[6]*pQrad[t]+pCf[7]*pQrad[t-1]+pCf[8]*pQrad[t-2]                #current and past Q radiative
        +pCf[9]*(pQconv[t]-vQhvac[t])+pCf[10]*(pQconv[t-1]-vQhvac[t-1])+pCf[11]*(pQconv[t-2]-vQhvac[t-2])           #current and past Q convective
        +pCf[12]*pQwind[t]+pCf[13]*pQwind[t-1]+pCf[14]*pQwind[t-2]          #current and past Q window solar
        +pCf[15]*pQsurf[t]+pCf[16]*pQsurf[t-1]+pCf[17]*pQsurf[t-2]          #current and past Q surface solar
        +pCf[18]*pTground[t]+pCf[19]*pTground[t-1]+pCf[20]*pTground[t-2]) #current and past ground temp
    